# Demand Forecasting Project - 改进版
## Data Science Internship Project

### 项目流程

**第一周： 数据处理**
1. 数据加载
2. 探索性数据分析（EDA）
3. 离群值检测和清理
4. 数据预处理
5. 特征工程
6. 数据拆分和验证

**第二周： 模型训练**
1. 模型训练
2. 模型评估
3. 模型调参
4. 模型预测
5. 模型保存和加载
6. 模型部署

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

## 1. 数据加载

In [ ]:
# Load training and test data
train_data = pd.read_csv('../archive/train_0irEZ2H.csv')
test_data = pd.read_csv('../archive/test_nfaJ3J5.csv')

print("Training data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

print("\nTraining data preview:")
display(train_data.head())

print("\nTest data preview:")
display(test_data.head())

In [ ]:
# Check data types and basic info
print("Training data info:")
train_data.info()

print("\nTest data info:")
test_data.info()

## 2. 探索性数据分析（EDA）

In [ ]:
print("Training data description:")
display(train_data.describe())

print("Test data description:")
display(test_data.describe())

In [ ]:
# Check for missing values
print("Missing values in training data:")
missing_train = train_data.isnull().sum()
print(missing_train[missing_train > 0])

print("\nMissing values in test data:")
missing_test = test_data.isnull().sum()
print(missing_test[missing_test > 0])

In [ ]:
# Convert week column to datetime
train_data['week'] = pd.to_datetime(train_data['week'], format='%y/%m/%d')
test_data['week'] = pd.to_datetime(test_data['week'], format='%y/%m/%d')

print("Date range in training data:")
print(f"Start: {train_data['week'].min()}, End: {train_data['week'].max()}")

print("\nDate range in test data:")
print(f"Start: {test_data['week'].min()}, End: {test_data['week'].max()}")

In [ ]:
# Distribution of target variable (units_sold)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(train_data['units_sold'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Units Sold')
plt.xlabel('Units Sold')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.boxplot(train_data['units_sold'])
plt.title('Boxplot of Units Sold')
plt.ylabel('Units Sold')

plt.tight_layout()
plt.show()

print(f"Statistics for units_sold:")
print(f"Mean: {train_data['units_sold'].mean():.2f}")
print(f"Median: {train_data['units_sold'].median():.2f}")
print(f"Std: {train_data['units_sold'].std():.2f}")
print(f"Min: {train_data['units_sold'].min()}")
print(f"Max: {train_data['units_sold'].max()}")

In [ ]:
# Price analysis
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(train_data['total_price'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Total Price')
plt.xlabel('Total Price')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(train_data['base_price'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Base Price')
plt.xlabel('Base Price')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
price_diff = train_data['base_price'] - train_data['total_price']
plt.hist(price_diff, bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Price Difference')
plt.xlabel('Base Price - Total Price')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Categorical variables analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Store analysis
store_counts = train_data['store_id'].value_counts()
axes[0, 0].bar(store_counts.index, store_counts.values)
axes[0, 0].set_title('Number of Records per Store')
axes[0, 0].set_xlabel('Store ID')
axes[0, 0].set_ylabel('Count')

# SKU analysis
sku_counts = train_data['sku_id'].value_counts()
axes[0, 1].hist(sku_counts.values, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution of Records per SKU')
axes[0, 1].set_xlabel('Number of Records per SKU')
axes[0, 1].set_ylabel('Frequency')

# Promotion features
featured_counts = train_data['is_featured_sku'].value_counts()
axes[1, 0].bar(featured_counts.index, featured_counts.values)
axes[1, 0].set_title('Featured SKU Distribution')
axes[1, 0].set_xlabel('Is Featured (0/1)')
axes[1, 0].set_ylabel('Count')

display_counts = train_data['is_display_sku'].value_counts()
axes[1, 1].bar(display_counts.index, display_counts.values)
axes[1, 1].set_title('Display SKU Distribution')
axes[1, 1].set_xlabel('Is Displayed (0/1)')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

## 3. 异常值检测与清洗

In [ ]:
# Identify outliers in units_sold using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

outliers, lower_bound, upper_bound = detect_outliers_iqr(train_data, 'units_sold')

print(f"Lower bound for outliers: {lower_bound:.2f}")
print(f"Upper bound for outliers: {upper_bound:.2f}")
print(f"Number of outliers detected: {len(outliers)}")
print(f"Percentage of outliers: {len(outliers)/len(train_data)*100:.2f}%")

In [ ]:
# Visualize outliers
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(train_data['units_sold'], bins=100, edgecolor='black', alpha=0.7)
plt.axvline(lower_bound, color='r', linestyle='--', label=f'Lower bound ({lower_bound:.2f})')
plt.axvline(upper_bound, color='r', linestyle='--', label=f'Upper bound ({upper_bound:.2f})')
plt.title('Units Sold Distribution with Outlier Bounds')
plt.xlabel('Units Sold')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(train_data['units_sold'])
plt.title('Boxplot of Units Sold')
plt.ylabel('Units Sold')

plt.tight_layout()
plt.show()

In [ ]:
# Check outliers in price variables
price_outliers_total, _, _ = detect_outliers_iqr(train_data, 'total_price')
price_outliers_base, _, _ = detect_outliers_iqr(train_data, 'base_price')

print(f"Outliers in total_price: {len(price_outliers_total)} ({len(price_outliers_total)/len(train_data)*100:.2f}%)")
print(f"Outliers in base_price: {len(price_outliers_base)} ({len(price_outliers_base)/len(train_data)*100:.2f}%)")

## 4. 数据预处理

In [ ]:
# Handle missing values
# Check the specific missing value in total_price
missing_price_rows = train_data[train_data['total_price'].isnull()]
print("Rows with missing total_price in training data:")
display(missing_price_rows)

# Impute missing value with base_price (reasonable assumption when no discount)
train_data['total_price'].fillna(train_data['base_price'], inplace=True)
print("\nMissing values after imputation:")
print(train_data.isnull().sum())

In [ ]:
# Check for duplicates
duplicates_train = train_data.duplicated().sum()
duplicates_test = test_data.duplicated().sum()

print(f"Duplicate rows in training data: {duplicates_train}")
print(f"Duplicate rows in test data: {duplicates_test}")

# Remove duplicates if any
if duplicates_train > 0:
    train_data = train_data.drop_duplicates()
    print(f"Removed {duplicates_train} duplicate rows from training data")
    
if duplicates_test > 0:
    test_data = test_data.drop_duplicates()
    print(f"Removed {duplicates_test} duplicate rows from test data")

In [ ]:
# Data type optimization
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            if df[col].min() >= 0 and df[col].max() <= 255:
                df[col] = df[col].astype('uint8')
            elif df[col].min() >= -128 and df[col].max() <= 127:
                df[col] = df[col].astype('int8')
            elif df[col].min() >= 0 and df[col].max() <= 65535:
                df[col] = df[col].astype('uint16')
            elif df[col].min() >= -32768 and df[col].max() <= 32767:
                df[col] = df[col].astype('int16')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

# Optimize data types
initial_memory_train = train_data.memory_usage(deep=True).sum() / 1024**2
initial_memory_test = test_data.memory_usage(deep=True).sum() / 1024**2

train_data = optimize_dtypes(train_data)
test_data = optimize_dtypes(test_data)

final_memory_train = train_data.memory_usage(deep=True).sum() / 1024**2
final_memory_test = test_data.memory_usage(deep=True).sum() / 1024**2

print(f"Training data memory reduced from {initial_memory_train:.2f} MB to {final_memory_train:.2f} MB ({(initial_memory_train-final_memory_train)/initial_memory_train*100:.1f}% reduction)")
print(f"Test data memory reduced from {initial_memory_test:.2f} MB to {final_memory_test:.2f} MB ({(initial_memory_test-final_memory_test)/initial_memory_test*100:.1f}% reduction)")

## 5. 特征工程

In [ ]:
# Create time-based features
def create_time_features(df):
    df = df.copy()
    df['year'] = df['week'].dt.year
    df['month'] = df['week'].dt.month
    df['day_of_week'] = df['week'].dt.dayofweek
    df['day_of_month'] = df['week'].dt.day
    df['quarter'] = df['week'].dt.quarter
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    return df

train_data = create_time_features(train_data)
test_data = create_time_features(test_data)

print("Time-based features created successfully!")

In [ ]:
# Create price-based features
def create_price_features(df):
    df = df.copy()
    # Price difference and discount ratio
    df['price_discount'] = df['base_price'] - df['total_price']
    df['discount_ratio'] = df['price_discount'] / (df['base_price'] + 1e-8)  # Adding small epsilon to avoid division by zero
    
    # Log transformations
    df['log_total_price'] = np.log1p(df['total_price'])
    df['log_base_price'] = np.log1p(df['base_price'])
    
    # Price ratios
    df['price_ratio'] = df['total_price'] / (df['base_price'] + 1e-8)
    
    return df

train_data = create_price_features(train_data)
test_data = create_price_features(test_data)

print("Price-based features created successfully!")

In [ ]:
# Create promotion features
def create_promotion_features(df):
    df = df.copy()
    # Combined promotion flag
    df['is_promoted'] = ((df['is_featured_sku'] == 1) | (df['is_display_sku'] == 1)).astype(int)
    
    # Promotion intensity
    df['promotion_intensity'] = df['is_featured_sku'] + df['is_display_sku']
    
    return df

train_data = create_promotion_features(train_data)
test_data = create_promotion_features(test_data)

print("Promotion features created successfully!")

In [ ]:
# 添加滞后特征：过去1周、2周、4周的销量数据（仅用于训练数据）
def create_lag_features_train(df):
    df = df.copy()
    
    # 检查是否包含units_sold列
    if 'units_sold' not in df.columns:
        print("警告：数据中不包含units_sold列，无法创建滞后特征")
        return df
    
    # 创建一个用于计算滞后特征的数据框
    # 需要按store_id和sku_id分组来计算滞后特征
    lag_df = df[['week', 'store_id', 'sku_id', 'units_sold']].copy()
    
    # 按store_id和sku_id分组，按日期排序
    lag_df = lag_df.sort_values(['store_id', 'sku_id', 'week'])
    
    # 计算滞后特征（1周、2周、4周前的销量）
    lag_df['lag_1w'] = lag_df.groupby(['store_id', 'sku_id'])['units_sold'].shift(1)
    lag_df['lag_2w'] = lag_df.groupby(['store_id', 'sku_id'])['units_sold'].shift(2)
    lag_df['lag_4w'] = lag_df.groupby(['store_id', 'sku_id'])['units_sold'].shift(4)
    
    # 将滞后特征合并回原数据框
    df = df.merge(lag_df[['week', 'store_id', 'sku_id', 'lag_1w', 'lag_2w', 'lag_4w']], 
                  on=['week', 'store_id', 'sku_id'], how='left')
    
    return df

# 为测试数据创建不依赖于units_sold的滞后特征函数
def create_lag_features_test(df):
    df = df.copy()
    print("测试数据不创建依赖于units_sold的滞后特征")
    return df

# 应用滞后特征
print("正在创建滞后特征...")
train_data = create_lag_features_train(train_data)
test_data = create_lag_features_test(test_data)

# 检查新特征
print("滞后特征创建完成！")
print("新增的滞后特征列:")
new_cols = [col for col in train_data.columns if col.startswith('lag_')]
print(new_cols)

# 查看包含滞后特征的样本数据
print("\n包含滞后特征的样本数据:")
display(train_data[['week', 'store_id', 'sku_id', 'units_sold', 'lag_1w', 'lag_2w', 'lag_4w']].head(10))

In [ ]:
# 添加滚动统计特征：计算过去一段时间的销量均值、标准差等统计量（仅用于训练数据）
def create_rolling_features_train(df):
    df = df.copy()
    
    # 检查是否包含units_sold列
    if 'units_sold' not in df.columns:
        print("警告：数据中不包含units_sold列，无法创建滚动统计特征")
        return df
    
    # 创建一个用于计算滚动统计特征的数据框
    rolling_df = df[['week', 'store_id', 'sku_id', 'units_sold']].copy()
    
    # 按store_id和sku_id分组，按日期排序
    rolling_df = rolling_df.sort_values(['store_id', 'sku_id', 'week'])
    
    # 计算滚动统计特征（过去4周的均值和标准差）
    rolling_df['rolling_mean_4w'] = rolling_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=4, min_periods=1).mean())
    rolling_df['rolling_std_4w'] = rolling_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=4, min_periods=1).std())
    
    # 计算过去8周的均值和标准差
    rolling_df['rolling_mean_8w'] = rolling_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=8, min_periods=1).mean())
    rolling_df['rolling_std_8w'] = rolling_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=8, min_periods=1).std())
    
    # 将滚动统计特征合并回原数据框
    df = df.merge(rolling_df[['week', 'store_id', 'sku_id', 'rolling_mean_4w', 'rolling_std_4w', 
                              'rolling_mean_8w', 'rolling_std_8w']], 
                  on=['week', 'store_id', 'sku_id'], how='left')
    
    return df

# 为测试数据创建不依赖于units_sold的滚动统计特征函数
def create_rolling_features_test(df):
    df = df.copy()
    print("测试数据不创建依赖于units_sold的滚动统计特征")
    return df

# 应用滚动统计特征
print("正在创建滚动统计特征...")
train_data = create_rolling_features_train(train_data)
test_data = create_rolling_features_test(test_data)

# 检查新特征
print("滚动统计特征创建完成！")
print("新增的滚动统计特征列:")
new_cols = [col for col in train_data.columns if col.startswith('rolling_')]
print(new_cols)

# 查看包含滚动统计特征的样本数据
print("\n包含滚动统计特征的样本数据:")
display(train_data[['week', 'store_id', 'sku_id', 'units_sold', 'rolling_mean_4w', 'rolling_std_4w', 
                    'rolling_mean_8w', 'rolling_std_8w']].head(10))

In [ ]:
# 添加同比环比特征：计算销量的增长率和变化趋势（仅用于训练数据）
def create_trend_features_train(df):
    df = df.copy()
    
    # 检查是否包含units_sold列
    if 'units_sold' not in df.columns:
        print("警告：数据中不包含units_sold列，无法创建趋势特征")
        return df
    
    # 创建一个用于计算趋势特征的数据框
    trend_df = df[['week', 'store_id', 'sku_id', 'units_sold']].copy()
    
    # 按store_id和sku_id分组，按日期排序
    trend_df = trend_df.sort_values(['store_id', 'sku_id', 'week'])
    
    # 计算环比增长率（与上周相比）
    trend_df['units_sold_lag1'] = trend_df.groupby(['store_id', 'sku_id'])['units_sold'].shift(1)
    trend_df['week-over-week_growth'] = (trend_df['units_sold'] - trend_df['units_sold_lag1']) / (trend_df['units_sold_lag1'] + 1e-8)
    
    # 计算销量变化趋势（使用线性回归斜率）
    def calculate_trend(series):
        if len(series) < 2:
            return 0
        x = np.arange(len(series))
        slope = np.polyfit(x, series, 1)[0] if len(series) > 1 else 0
        return slope
    
    trend_df['sales_trend_4w'] = trend_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=4, min_periods=1).apply(calculate_trend, raw=True))
    
    trend_df['sales_trend_8w'] = trend_df.groupby(['store_id', 'sku_id'])['units_sold'].transform(
        lambda x: x.rolling(window=8, min_periods=1).apply(calculate_trend, raw=True))
    
    # 将趋势特征合并回原数据框
    df = df.merge(trend_df[['week', 'store_id', 'sku_id', 'week-over-week_growth', 'sales_trend_4w', 'sales_trend_8w']], 
                  on=['week', 'store_id', 'sku_id'], how='left')
    
    return df

# 为测试数据创建不依赖于units_sold的趋势特征函数
def create_trend_features_test(df):
    df = df.copy()
    print("测试数据不创建依赖于units_sold的趋势特征")
    return df

# 应用趋势特征
print("正在创建同比环比特征...")
train_data = create_trend_features_train(train_data)
test_data = create_trend_features_test(test_data)

# 检查新特征
print("同比环比特征创建完成！")
print("新增的趋势特征列:")
new_cols = [col for col in train_data.columns if 'growth' in col or 'trend' in col]
print(new_cols)

# 查看包含趋势特征的样本数据
print("\n包含趋势特征的样本数据:")
display(train_data[['week', 'store_id', 'sku_id', 'units_sold', 'week-over-week_growth', 'sales_trend_4w', 'sales_trend_8w']].head(10))

In [ ]:
# 添加商品/店铺历史表现：添加商品和店铺的历史平均销量等特征
def create_historical_features_train(df):
    df = df.copy()
    
    # 检查是否包含units_sold列
    if 'units_sold' not in df.columns:
        print("警告：数据中不包含units_sold列，无法创建历史表现特征")
        return df
    
    # 计算每个商品(sku_id)的历史平均销量、最大销量、最小销量
    sku_stats = df.groupby('sku_id')['units_sold'].agg(['mean', 'std', 'min', 'max']).reset_index()
    sku_stats.columns = ['sku_id', 'sku_avg_sales', 'sku_std_sales', 'sku_min_sales', 'sku_max_sales']
    
    # 计算每个店铺(store_id)的历史平均销量、最大销量、最小销量
    store_stats = df.groupby('store_id')['units_sold'].agg(['mean', 'std', 'min', 'max']).reset_index()
    store_stats.columns = ['store_id', 'store_avg_sales', 'store_std_sales', 'store_min_sales', 'store_max_sales']
    
    # 将历史统计特征合并回原数据框
    df = df.merge(sku_stats, on='sku_id', how='left')
    df = df.merge(store_stats, on='store_id', how='left')
    
    # 计算商品和店铺的销量占比特征
    df['sku_sales_ratio'] = df['units_sold'] / (df['sku_avg_sales'] + 1e-8)
    df['store_sales_ratio'] = df['units_sold'] / (df['store_avg_sales'] + 1e-8)
    
    return df

def create_historical_features_test(df, train_df):
    df = df.copy()
    
    # 使用训练数据的统计信息
    sku_stats = train_df.groupby('sku_id')['units_sold'].agg(['mean', 'std', 'min', 'max']).reset_index()
    sku_stats.columns = ['sku_id', 'sku_avg_sales', 'sku_std_sales', 'sku_min_sales', 'sku_max_sales']
    
    store_stats = train_df.groupby('store_id')['units_sold'].agg(['mean', 'std', 'min', 'max']).reset_index()
    store_stats.columns = ['store_id', 'store_avg_sales', 'store_std_sales', 'store_min_sales', 'store_max_sales']
    
    # 将历史统计特征合并到测试数据
    df = df.merge(sku_stats, on='sku_id', how='left')
    df = df.merge(store_stats, on='store_id', how='left')
    
    # 计算商品和店铺的销量占比特征（使用历史平均值）
    df['sku_sales_ratio'] = df['sku_avg_sales'] / (df['sku_avg_sales'] + 1e-8)  # 这里使用历史平均值
    df['store_sales_ratio'] = df['store_avg_sales'] / (df['store_avg_sales'] + 1e-8)
    
    return df

# 应用历史表现特征
print("正在创建商品/店铺历史表现特征...")
train_data = create_historical_features_train(train_data)
test_data = create_historical_features_test(test_data, train_data)

# 检查新特征
print("商品/店铺历史表现特征创建完成！")
print("新增的历史表现特征列:")
new_cols = [col for col in train_data.columns if 'sku_' in col or 'store_' in col]
print(new_cols)

# 查看包含历史表现特征的样本数据
print("\n包含历史表现特征的样本数据:")
display(train_data[['week', 'store_id', 'sku_id', 'units_sold', 'sku_avg_sales', 'store_avg_sales', 
                    'sku_sales_ratio', 'store_sales_ratio']].head(10))

In [ ]:
# 添加交叉特征：创建促销与时间等因素的交叉特征
def create_interaction_features(df):
    df = df.copy()
    
    # 促销与时间的交叉特征
    # 促销类型与星期的交叉
    df['featured_weekday'] = df['is_featured_sku'] * df['day_of_week']
    df['display_weekday'] = df['is_display_sku'] * df['day_of_week']
    
    # 促销组合与月份的交叉
    df['promoted_month'] = df['is_promoted'] * df['month']
    
    # 价格折扣与促销的交叉
    df['discount_featured'] = df['discount_ratio'] * df['is_featured_sku']
    df['discount_display'] = df['discount_ratio'] * df['is_display_sku']
    
    # 价格与星期的交叉
    df['price_weekday'] = df['total_price'] * df['day_of_week']
    
    return df

# 应用交叉特征
print("正在创建交叉特征...")
train_data = create_interaction_features(train_data)
test_data = create_interaction_features(test_data)

# 检查新特征
print("交叉特征创建完成！")
print("新增的交叉特征列:")
interaction_cols = ['featured_weekday', 'display_weekday', 'promoted_month', 'discount_featured', 
                   'discount_display', 'price_weekday']
print(interaction_cols)

# 查看包含交叉特征的样本数据
print("\n包含交叉特征的样本数据:")
display(train_data[['week', 'store_id', 'sku_id', 'units_sold', 'is_featured_sku', 'is_display_sku',
                    'day_of_week', 'month', 'discount_ratio', 'total_price'] + interaction_cols].head(10))

In [ ]:
# Correlation analysis of engineered features with target
feature_cols = ['total_price', 'base_price', 'is_featured_sku', 'is_display_sku', 'price_discount', 'discount_ratio', 'log_total_price', 'log_base_price', 'is_promoted', 'promotion_intensity', 'month', 'day_of_week']

correlation_data = train_data[feature_cols + ['units_sold']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Matrix of Features with Target Variable')
plt.show()

## 6. 数据质量验证

In [ ]:
# Final dataset summary and data quality validation
print("=== FINAL DATASET SUMMARY ===")
print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

# 显示所有特征列（不包括record_ID和目标变量）
train_features = [col for col in train_data.columns if col not in ['record_ID', 'week', 'store_id', 'sku_id', 'units_sold']]
test_features = [col for col in test_data.columns if col not in ['record_ID', 'week', 'store_id', 'sku_id']]
print(f"\nTraining features: {len(train_features)}")
print(f"Test features: {len(test_features)}")

# 检查特征一致性
common_features = set(train_features) & set(test_features)
train_only_features = set(train_features) - set(test_features)
test_only_features = set(test_features) - set(train_features)
print(f"Common features: {len(common_features)}")
print(f"Training-only features: {len(train_only_features)}")
print(f"Test-only features: {len(test_only_features)}")

if train_only_features:
    print(f"\nTraining-only features (should be mostly target-dependent): {train_only_features}")
    
if test_only_features:
    print(f"\nTest-only features: {test_only_features}")

# 检查缺失值
print("\n=== MISSING VALUES CHECK ===")
train_missing = train_data[train_features].isnull().sum()
test_missing = test_data[test_features].isnull().sum()
print(f"Missing values in training data: {train_missing.sum()}")
print(f"Missing values in test data: {test_missing.sum()}")

# 按类别分组显示特征
time_features = [col for col in train_features if col in ['year', 'month', 'day_of_week', 'day_of_month', 'quarter', 'is_weekend']]
price_features = [col for col in train_features if col in ['total_price', 'base_price', 'price_discount', 'discount_ratio', 'log_total_price', 'log_base_price', 'price_ratio']]
promotion_features = [col for col in train_features if col in ['is_featured_sku', 'is_display_sku', 'is_promoted', 'promotion_intensity']]
lag_features = [col for col in train_features if col.startswith('lag_')]
rolling_features = [col for col in train_features if col.startswith('rolling_')]
trend_features = [col for col in train_features if 'growth' in col or 'trend' in col]
historical_features = [col for col in train_features if col.startswith('sku_') or col.startswith('store_')]
interaction_features = [col for col in train_features if col in ['featured_weekday', 'display_weekday', 'promoted_month', 'discount_featured', 'discount_display', 'price_weekday']]

print("\n=== FEATURE CATEGORIES ===")
print(f"Time-based features ({len(time_features)}): {time_features}")
print(f"Price-based features ({len(price_features)}): {price_features}")
print(f"Promotion features ({len(promotion_features)}): {promotion_features}")
print(f"Lag features ({len(lag_features)}): {lag_features}")
print(f"Rolling statistics features ({len(rolling_features)}): {rolling_features}")
print(f"Trend features ({len(trend_features)}): {trend_features}")
print(f"Historical performance features ({len(historical_features)}): {historical_features}")
print(f"Interaction features ({len(interaction_features)}): {interaction_features}")

# 保存处理后的数据（移除record_ID列）
train_processed = train_data.drop(['record_ID'], axis=1)
test_processed = test_data.drop(['record_ID'], axis=1)

train_processed.to_csv('../archive/train_processed_improved.csv', index=False)
test_processed.to_csv('../archive/test_processed_improved.csv', index=False)
print("\nProcessed data saved to archive folder.")
print("File names: train_processed_improved.csv, test_processed_improved.csv")

## 总结

完成了改进版的数据分析pipeline，并实现了高级特征工程：

1. **数据加载**：加载训练和测试数据集
2. **探索性数据分析**：分析分布、相关性和模式
3. **异常值检测**：使用IQR方法识别异常值
4. **数据预处理**：处理缺失值、删除重复项、优化数据类型
5. **基础特征工程**：创建了基于时间、基于价格、基于促销的特征
6. **高级特征工程**：
   - 滞后特征：过去1周、2周、4周的销量数据（仅用于训练数据）
   - 滚动统计特征：过去一段时间的销量均值、标准差等统计量（仅用于训练数据）
   - 同比环比特征：销量的增长率和变化趋势（仅用于训练数据）
   - 商品/店铺历史表现：商品和店铺的历史平均销量等特征
   - 交叉特征：促销与时间等因素的交叉特征
7. **数据质量验证**：检查特征一致性、缺失值等

### 改进点
1. **数据一致性**：确保训练集和测试集具有相同的特征列（除了目标变量相关的特征）
2. **时间序列处理**：正确处理时间序列特征，避免数据泄露
3. **错误处理**：添加了适当的错误检查和处理机制
4. **数据验证**：添加了数据质量验证步骤
5. **特征清理**：移除了无用的record_ID列

接下来可以进行机器学习建模...